In [1]:
# Import libaries needed for Interactive Maps
# !pip install folium wget OR %pip install folium wget for INSIDE current kernel

%pip install folium wget
import folium
import pandas as pd
import wget

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load pre-set dataset (2_data_wrangling.csv)
df = pd.read_csv('2_data_wrangling.csv')
print("Dataset Loaded for Mapping")

Dataset Loaded for Mapping


In [3]:
# Print Top 5 rows of the dataset to check import
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,6,2010-06-04T18:45:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0003,-80.577366,28.561857,0
1,7,2010-12-08T15:43:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0004,-80.577366,28.561857,0
2,8,2012-05-22T07:44:00.000Z,Falcon 9,525.0,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0005,-80.577366,28.561857,0
3,9,2012-10-08T00:35:00.000Z,Falcon 9,400.0,ISS,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0006,-80.577366,28.561857,0
4,10,2013-03-01T19:10:00.000Z,Falcon 9,677.0,ISS,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0007,-80.577366,28.561857,0


In [24]:
# Set the map focus to NASA Kennedy Space Center (Visitor Complex) using google maps co-ordinates (Coordinates: 28.5729, -80.6490)
ksc_coordinate = [28.5729, -80.6490]

#Setting the site_map to match Kennedy Space Center and setting zoom to 5 to see the other relative launch sites)
site_map = folium.Map(location=ksc_coordinate, zoom_start=3.5)

# Show map (will be empty)
site_map

In [17]:
# Apply markers (red circles) to Dataset Launch Sites
launch_sites_df = df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]

# Add a Red Circle for each site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Latitude'], row['Longitude']]
    
    # Add the circle marker (Visual dot)
    folium.Circle(
        coordinate,
        radius=1000, 
        color='#d35400', # set colour to red
        fill=True
    ).add_child(folium.Popup(row['LaunchSite'])).add_to(site_map)

    # Add the text label
    folium.map.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'],
            )
        ).add_to(site_map)

#Show map with launch sites labeled
site_map

In [25]:
# Function to assign color based on Class (1=Success, 0=Fail)
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Apply this function to create a new column 'marker_color'
df['marker_color'] = df['Class'].apply(assign_marker_color)

# Check if it worked by showing 10 rows to see marker_color
print(df[['LaunchSite', 'Class', 'marker_color']].head(10))

     LaunchSite  Class marker_color
0  CCSFS SLC 40      0          red
1  CCSFS SLC 40      0          red
2  CCSFS SLC 40      0          red
3  CCSFS SLC 40      0          red
4  CCSFS SLC 40      0          red
5   VAFB SLC 4E      0          red
6  CCSFS SLC 40      0          red
7  CCSFS SLC 40      0          red
8  CCSFS SLC 40      1        green
9  CCSFS SLC 40      1        green


In [29]:
# Import MarkerCluster and Element tool for marking and captions
from folium.plugins import MarkerCluster
from folium import Element

# Create a base map
ksc_coordinate = [28.5729, -80.6490]
site_map = folium.Map(location=ksc_coordinate, zoom_start=3.5)

# Add a Title (HTML Caption)
title_html = '''
     <h3 align="center" style="font-size:20px"><b>"Click cluster to see" SpaceX Launch Outcomes (Green=Success, Red=Fail)</b></h3>
             '''
site_map.get_root().html.add_child(Element(title_html))

# Create the Marker Cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through the dataframe and add Markers
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=row['BoosterVersion'],  # Shows on Click
        tooltip=f"{row['BoosterVersion']} (Click for info)" # Shows on Hover
    ).add_to(marker_cluster)

# Show the visualised and marked map
site_map

In [ ]:
# Save the map as an interactive HTML file
site_map.save("launch_site_map.html")
print("Map saved as 'launch_site_map.html'. Check your folder!")